In [1]:
import librosa
import librosa.display
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from IPython.display import Audio
%matplotlib inline
sns.set_style("ticks")

In [2]:
def plot_stft(sig):
    D = librosa.amplitude_to_db(np.abs(librosa.stft(sig)), ref=np.max)
    librosa.display.specshow(D, y_axis='linear');

In [3]:
def plot_mel_specgtrogram(y):
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(S_dB, x_axis='time',
                             y_axis='mel', sr=sr);



In [4]:
def plot_dft(sig, win_len=1024):
    window = np.hamming(win_len)
    sig = window * sig[:win_len]
    D = np.abs(np.fft.fft(sig))[:win_len//2]
    xs = np.arange(win_len/2) * (22050/win_len)
    plt.plot(xs, D)

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score

In [6]:
def extract_mfccs(y, sr):
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    return np.concatenate([
        mfccs.mean(axis=1),
        mfccs.std(axis=1)
    ])

def extract_chroma(y, sr):
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    return np.concatenate([
        chroma.mean(axis=1),
        chroma.std(axis=1)
    ])

def extract_feats(y, sr):
    chroma = extract_chroma(y, sr)
    mfccs = extract_mfccs(y, sr)
    feats = np.concatenate([chroma, mfccs])
    
    labels = []   
    for i in range(len(chroma) // 2):
        labels.append('chroma_{}_mean'.format(i))
    for i in range(len(chroma) // 2):
        labels.append('chroma_{}_std'.format(i))
    for i in range(len(mfccs) // 2):
        labels.append('mfcc_{}_mean'.format(i))
    for i in range(len(mfccs) // 2):
        labels.append('mfcc_{}_std'.format(i))
        
    return feats, labels

def feats_for_fname(fname):
    y, sr = librosa.core.load(fname)
    return extract_feats(y, sr)

def fname_generator(genre):
    #for root, dirs, files in os.walk('/media/bita/Elements/Ironhack-Final/genres'):
    for root, dirs, files in os.walk('/media/bita/Elements/Ironhack-Final/Audios/genres'):    
        for file in files:
            if file.endswith('.wav') and genre in file:
                yield os.path.join(root, file)         
                
def df_for_genre(genre):
    fnames = []
    feats = []
    for fname in fname_generator(genre):
        # print("extracting feats for {}".format(fname))
        cur_feats, labels = feats_for_fname(fname)
        fnames.append(fname)
        feats.append(cur_feats)
    df = pd.DataFrame(feats, columns=labels, index=fnames)
    df['label'] = genre
    return df

In [7]:
#data = pd.concat([df_for_genre(genre) for genre in ['pop', 'classical']])
#data = pd.concat([df_for_genre(genre) for genre in ['Meow', 'Bark']])
data = pd.concat([df_for_genre(genre) for genre in ['Cows', 'Keys','Fireworks']])

/home/bita/anaconda3/envs/backpack1/lib/python3.7/site-packages/librosa/core/pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [8]:
data

,chroma_0_mean,chroma_1_mean,chroma_2_mean,chroma_3_mean,chroma_4_mean,chroma_5_mean,chroma_6_mean,chroma_7_mean,chroma_8_mean,chroma_9_mean,...,mfcc_11_std,mfcc_12_std,mfcc_13_std,mfcc_14_std,mfcc_15_std,mfcc_16_std,mfcc_17_std,mfcc_18_std,mfcc_19_std,label
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.0e979b8c.wav,0.570928,0.545968,0.545207,0.557640,0.534938,0.512101,0.513369,0.552215,0.553365,0.490339,...,2.645489,3.971400,5.743928,6.686796,5.154151,3.746077,4.209829,2.047308,6.666462,Cows
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.0f36e49a.wav,0.440007,0.589444,0.830202,0.641879,0.523190,0.484395,0.677281,0.482703,0.398226,0.490785,...,4.287548,3.114848,4.069334,3.598865,2.089022,3.811758,5.182171,7.733828,5.552181,Cows
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.10efbc5b.wav,0.048046,0.019395,0.236221,0.858508,0.504872,0.191672,0.061350,0.029342,0.027880,0.035950,...,3.787081,4.713351,3.468052,3.868890,4.111562,5.233063,5.715356,5.681756,12.787819,Cows
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.11c90896.wav,0.412085,0.734992,0.882000,0.625896,0.422707,0.433195,0.279917,0.242717,0.258143,0.294563,...,5.703547,9.693671,5.774012,3.484506,4.906104,6.220828,3.637396,10.150738,9.125066,Cows
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.16eeb46e.wav,0.191871,0.195733,0.279468,0.176665,0.160090,0.161065,0.214525,0.241234,0.180280,0.449515,...,3.882898,7.169450,7.188643,7.773633,7.495047,7.658581,4.977293,4.856813,3.635543,Cows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.a067129f.wav,0.581382,0.620916,0.640869,0.671509,0.705806,0.717650,0.688187,0.654803,0.610219,0.610005,...,3.792722,4.612837,4.062277,3.854307,3.320247,3.555770,3.470971,3.438119,3.312329,Fireworks
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.a10dbb12.wav,0.677024,0.616257,0.564342,0.579165,0.526369,0.449601,0.472232,0.455095,0.411832,0.474112,...,5.702889,6.863624,4.583376,5.310613,5.931530,6.247245,6.928460,6.952342,6.873388,Fireworks
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.a2d8cc0e.wav,0.558839,0.538764,0.611471,0.659820,0.654872,0.683374,0.647561,0.635647,0.595826,0.581948,...,3.754612,4.108128,2.971141,3.943295,2.984417,3.618123,3.233097,4.283641,2.756314,Fireworks
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.a309ddda.wav,0.542230,0.489791,0.515602,0.542760,0.604374,0.670135,0.716142,0.737965,0.674290,0.628751,...,5.137615,3.855678,4.285246,3.098147,3.515209,3.003388,3.218521,2.512391,3.149167,Fireworks


In [9]:
y = data['label']
X = data.drop('label', axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=F

In [13]:
X_test

,chroma_0_mean,chroma_1_mean,chroma_2_mean,chroma_3_mean,chroma_4_mean,chroma_5_mean,chroma_6_mean,chroma_7_mean,chroma_8_mean,chroma_9_mean,...,mfcc_10_std,mfcc_11_std,mfcc_12_std,mfcc_13_std,mfcc_14_std,mfcc_15_std,mfcc_16_std,mfcc_17_std,mfcc_18_std,mfcc_19_std
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.47a42a02.wav,0.092618,0.284467,0.977677,0.413844,0.085687,0.089305,0.177844,0.316204,0.248887,0.148812,...,4.550761,5.378458,5.521772,7.115781,9.135624,8.498100,8.797886,7.646821,7.324828,7.724832
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.e1fd95d5.wav,0.465422,0.665239,0.827985,0.615140,0.487571,0.584431,0.646970,0.439825,0.420590,0.521275,...,6.403136,4.078032,2.711546,4.366212,3.955171,2.170880,2.707530,5.611818,8.286240,5.723875
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.8bdab359.wav,0.332188,0.347097,0.381455,0.406744,0.395012,0.422695,0.544981,0.772656,0.669191,0.366762,...,11.300592,8.182364,9.220284,7.670178,11.293714,4.737746,5.114260,7.764853,7.778929,9.214108
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.41eadcab.wav,0.645062,0.658593,0.680559,0.674959,0.644687,0.607254,0.518763,0.524759,0.514807,0.524012,...,5.486921,5.277484,4.599896,4.457453,4.242080,4.005628,4.054725,3.510299,3.673066,3.470926
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.0bb6b85a.wav,0.557546,0.585096,0.568882,0.689384,0.492487,0.285636,0.326099,0.407233,0.503166,0.566478,...,5.633112,5.318146,5.732742,4.906494,4.768893,5.237311,4.603494,4.542755,5.103574,4.637789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.09a585bf.wav,0.436692,0.605281,0.815952,0.419485,0.240556,0.255332,0.311801,0.260388,0.302690,0.450291,...,6.218552,6.567881,6.424757,5.431433,5.764111,5.268544,4.662983,4.631197,4.825700,5.344444
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.4430e0ee.wav,0.541432,0.536219,0.531971,0.498188,0.457303,0.468385,0.463778,0.601592,0.609511,0.511724,...,4.878267,2.325066,4.526315,5.617414,4.742563,3.932692,3.184304,3.477225,1.792192,4.558136
/media/bita/Elements/Ironhack-Final/Audios/genres/Test2/Keys.0b5ea11d.wav,0.673854,0.733070,0.625286,0.546502,0.289300,0.240569,0.339081,0.403566,0.359533,0.351177,...,5.011569,5.148346,4.730614,4.783234,4.776738,4.146475,4.177127,4.131035,4.187706,4.780869
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.2bfa6330.wav,0.546582,0.609013,0.707416,0.571030,0.432198,0.432535,0.408353,0.451072,0.659016,0.855632,...,4.425665,3.549971,4.179065,3.405408,5.400559,4.208577,3.758600,3.852561,3.803576,3.070877


In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
accuracy_score(y_test, y_pred)

0.8736842105263158

In [16]:
y_test

/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.47a42a02.wav                   Cows
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.e1fd95d5.wav                   Cows
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.8bdab359.wav    Fireworks
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.41eadcab.wav    Fireworks
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.0bb6b85a.wav                   Keys
                                                                                        ...    
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.09a585bf.wav                   Keys
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.4430e0ee.wav                   Cows
/media/bita/Elements/Ironhack-Final/Audios/genres/Test2/Keys.0b5ea11d.wav                  Keys
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.2bfa6330.wav                   Keys
/media/bita/Elements/Ironhack-Final/Audi

In [17]:
y_pred

array(['Keys', 'Cows', 'Fireworks', 'Fireworks', 'Keys', 'Cows', 'Cows',
       'Fireworks', 'Fireworks', 'Fireworks', 'Cows', 'Fireworks', 'Keys',
       'Fireworks', 'Fireworks', 'Keys', 'Cows', 'Cows', 'Cows',
       'Fireworks', 'Cows', 'Keys', 'Keys', 'Fireworks', 'Fireworks',
       'Cows', 'Cows', 'Fireworks', 'Fireworks', 'Cows', 'Fireworks',
       'Cows', 'Fireworks', 'Fireworks', 'Fireworks', 'Fireworks',
       'Fireworks', 'Fireworks', 'Fireworks', 'Fireworks', 'Cows', 'Cows',
       'Keys', 'Fireworks', 'Fireworks', 'Keys', 'Keys', 'Fireworks',
       'Cows', 'Fireworks', 'Fireworks', 'Keys', 'Fireworks', 'Fireworks',
       'Fireworks', 'Cows', 'Cows', 'Fireworks', 'Keys', 'Fireworks',
       'Cows', 'Fireworks', 'Keys', 'Cows', 'Fireworks', 'Fireworks',
       'Cows', 'Fireworks', 'Fireworks', 'Keys', 'Cows', 'Fireworks',
       'Keys', 'Cows', 'Fireworks', 'Cows', 'Cows', 'Cows', 'Keys',
       'Cows', 'Cows', 'Cows', 'Fireworks', 'Fireworks', 'Cows', 'Cows',
       'C

In [18]:
y_pred != y_test


/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.47a42a02.wav               True
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.e1fd95d5.wav              False
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.8bdab359.wav    False
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.41eadcab.wav    False
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.0bb6b85a.wav              False
                                                                                      ...  
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.09a585bf.wav              False
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.4430e0ee.wav              False
/media/bita/Elements/Ironhack-Final/Audios/genres/Test2/Keys.0b5ea11d.wav             False
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.2bfa6330.wav              False
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.09a7dc35.wav        

In [19]:
y_pred_test_wrong = (y_pred != y_test)
y_pred_test_right = (y_pred == y_test)

In [20]:
type(y_pred_test_wrong)

pandas.core.series.Series

In [21]:
df = y_pred_test_wrong.to_frame()

In [22]:
pd.options.display.max_rows
pd.set_option('display.max_rows', None)

df


,label
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.47a42a02.wav,True
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.e1fd95d5.wav,False
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.8bdab359.wav,False
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.41eadcab.wav,False
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.0bb6b85a.wav,False
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.cbcc8197.wav,False
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.58da3a95.wav,False
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.f63fc55a.wav,False
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.49a0c96f.wav,True
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.8d7e73e0.wav,False


In [23]:
# PlAY True/FALSE identifications
for t in df.index[0:1]:
    print(t)
    y, sr = librosa.core.load(t)
    
Audio(data=y, rate=sr)

/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.47a42a02.wav


In [24]:
df_wrong = y_pred_test_wrong.to_frame()
df_right = y_pred_test_right.to_frame()

In [25]:
X_test

,chroma_0_mean,chroma_1_mean,chroma_2_mean,chroma_3_mean,chroma_4_mean,chroma_5_mean,chroma_6_mean,chroma_7_mean,chroma_8_mean,chroma_9_mean,...,mfcc_10_std,mfcc_11_std,mfcc_12_std,mfcc_13_std,mfcc_14_std,mfcc_15_std,mfcc_16_std,mfcc_17_std,mfcc_18_std,mfcc_19_std
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.47a42a02.wav,0.092618,0.284467,0.977677,0.413844,0.085687,0.089305,0.177844,0.316204,0.248887,0.148812,...,4.550761,5.378458,5.521772,7.115781,9.135624,8.498100,8.797886,7.646821,7.324828,7.724832
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.e1fd95d5.wav,0.465422,0.665239,0.827985,0.615140,0.487571,0.584431,0.646970,0.439825,0.420590,0.521275,...,6.403136,4.078032,2.711546,4.366212,3.955171,2.170880,2.707530,5.611818,8.286240,5.723875
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.8bdab359.wav,0.332188,0.347097,0.381455,0.406744,0.395012,0.422695,0.544981,0.772656,0.669191,0.366762,...,11.300592,8.182364,9.220284,7.670178,11.293714,4.737746,5.114260,7.764853,7.778929,9.214108
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.41eadcab.wav,0.645062,0.658593,0.680559,0.674959,0.644687,0.607254,0.518763,0.524759,0.514807,0.524012,...,5.486921,5.277484,4.599896,4.457453,4.242080,4.005628,4.054725,3.510299,3.673066,3.470926
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.0bb6b85a.wav,0.557546,0.585096,0.568882,0.689384,0.492487,0.285636,0.326099,0.407233,0.503166,0.566478,...,5.633112,5.318146,5.732742,4.906494,4.768893,5.237311,4.603494,4.542755,5.103574,4.637789
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.cbcc8197.wav,0.825048,0.675500,0.644602,0.649351,0.574464,0.543135,0.605017,0.763622,0.768305,0.639889,...,12.767130,5.095306,8.783082,4.457276,3.149979,3.447472,6.609109,3.365286,4.236958,4.020969
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.58da3a95.wav,0.414499,0.437516,0.444595,0.612929,0.920187,0.440381,0.350141,0.362938,0.370590,0.423960,...,7.660678,14.341952,6.320094,4.274746,6.014727,10.022768,6.076216,8.161916,5.809674,4.110452
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.f63fc55a.wav,0.586267,0.650862,0.672240,0.631427,0.607671,0.562710,0.541608,0.523954,0.565369,0.524552,...,8.065955,7.208903,6.459535,6.513250,5.773944,5.631942,5.806810,6.049668,7.241219,6.069617
/media/bita/Elements/Ironhack-Final/Audios/genres/Keys/Keys.49a0c96f.wav,0.602195,0.583961,0.611866,0.660342,0.642067,0.627018,0.636362,0.677597,0.758909,0.910545,...,4.430191,4.403671,3.503699,5.438123,3.872005,4.798270,3.353942,4.261822,3.351150,4.021413
/media/bita/Elements/Ironhack-Final/Audios/genres/Fireworks/Fireworks.8d7e73e0.wav,0.804172,0.830745,0.795991,0.791949,0.742751,0.711163,0.703665,0.717967,0.732757,0.746438,...,3.488251,3.186998,2.769204,2.204643,3.123434,2.806131,3.155886,2.039025,2.467547,2.521703


In [ ]:
data = pd.concat([df_for_genre(genre) for genre in ['Cows', 'Keys']])
y = data['label']
X = data.drop('label', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [27]:
X

,chroma_0_mean,chroma_1_mean,chroma_2_mean,chroma_3_mean,chroma_4_mean,chroma_5_mean,chroma_6_mean,chroma_7_mean,chroma_8_mean,chroma_9_mean,...,mfcc_10_std,mfcc_11_std,mfcc_12_std,mfcc_13_std,mfcc_14_std,mfcc_15_std,mfcc_16_std,mfcc_17_std,mfcc_18_std,mfcc_19_std
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.0e979b8c.wav,0.570928,0.545968,0.545207,0.557640,0.534938,0.512101,0.513369,0.552215,0.553365,0.490339,...,3.701275,2.645489,3.971400,5.743928,6.686796,5.154151,3.746077,4.209829,2.047308,6.666462
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.0f36e49a.wav,0.440007,0.589444,0.830202,0.641879,0.523190,0.484395,0.677281,0.482703,0.398226,0.490785,...,5.565233,4.287548,3.114848,4.069334,3.598865,2.089022,3.811758,5.182171,7.733828,5.552181
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.10efbc5b.wav,0.048046,0.019395,0.236221,0.858508,0.504872,0.191672,0.061350,0.029342,0.027880,0.035950,...,4.821989,3.787081,4.713351,3.468052,3.868890,4.111562,5.233063,5.715356,5.681756,12.787819
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.11c90896.wav,0.412085,0.734992,0.882000,0.625896,0.422707,0.433195,0.279917,0.242717,0.258143,0.294563,...,4.453197,5.703547,9.693671,5.774012,3.484506,4.906104,6.220828,3.637396,10.150738,9.125066
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.16eeb46e.wav,0.191871,0.195733,0.279468,0.176665,0.160090,0.161065,0.214525,0.241234,0.180280,0.449515,...,7.415944,3.882898,7.169450,7.188643,7.773633,7.495047,7.658581,4.977293,4.856813,3.635543
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.172d7f14.wav,0.561955,0.560142,0.497688,0.492304,0.515798,0.486138,0.478240,0.566308,0.504201,0.449283,...,3.578416,3.173978,5.369750,6.716822,5.071347,3.883863,3.340211,2.904474,2.500544,6.066441
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.172e4f11.wav,0.381649,0.606295,0.950267,0.605087,0.422635,0.352018,0.323333,0.305299,0.318006,0.343508,...,3.634035,4.119580,5.158055,3.753444,2.105723,1.805801,4.372720,5.832515,9.991510,8.913378
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.184ca1c7.wav,0.518475,0.516923,0.476834,0.469866,0.469357,0.448578,0.416525,0.570931,0.566567,0.421961,...,3.154553,2.906093,4.902675,6.970251,4.567768,3.813703,3.459315,3.951618,1.164823,3.121321
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.1ac81bb0.wav,0.533976,0.493924,0.506659,0.350331,0.171526,0.192391,0.199714,0.231116,0.305103,0.416651,...,9.780393,12.538993,13.618818,10.430340,10.539600,10.744136,7.498009,9.178326,7.552176,8.476257
/media/bita/Elements/Ironhack-Final/Audios/genres/Cows/Cows.1b5d4f7c.wav,0.242506,0.185437,0.176301,0.264526,0.561587,0.351268,0.331454,0.434896,0.574158,0.701278,...,10.766361,17.433968,13.718418,10.812380,7.506429,5.703159,8.077309,5.870289,8.170335,8.827370


In [47]:
dataz = pd.concat([df_for_genre(genre) for genre in ['Test']])

In [48]:
Zwithlable= dataz

In [49]:
Zwithlable

,chroma_0_mean,chroma_1_mean,chroma_2_mean,chroma_3_mean,chroma_4_mean,chroma_5_mean,chroma_6_mean,chroma_7_mean,chroma_8_mean,chroma_9_mean,...,mfcc_11_std,mfcc_12_std,mfcc_13_std,mfcc_14_std,mfcc_15_std,mfcc_16_std,mfcc_17_std,mfcc_18_std,mfcc_19_std,label
/media/bita/Elements/Ironhack-Final/Audios/genres/Test/1.Test.wav,0.579144,0.624621,0.668492,0.650194,0.642008,0.657012,0.673763,0.649138,0.659904,0.620277,...,4.967063,4.636360,5.298031,5.124247,4.578990,3.804608,4.492270,4.076270,4.078952,Test
/media/bita/Elements/Ironhack-Final/Audios/genres/Test/Test.wav,0.706214,0.581418,0.520377,0.527471,0.515273,0.496977,0.522292,0.551171,0.599805,0.561232,...,4.023371,4.826013,3.691178,3.807537,4.239256,3.805862,4.044842,4.161927,5.047316,Test
/media/bita/Elements/Ironhack-Final/Audios/genres/Test2/Test.wav,0.579144,0.624621,0.668492,0.650194,0.642008,0.657012,0.673763,0.649138,0.659904,0.620277,...,4.967063,4.636360,5.298031,5.124247,4.578990,3.804608,4.492270,4.076270,4.078952,Test


In [51]:
Z = Zwithlable.drop('label', axis=1)


In [52]:
z_pred = pipeline.predict(Z)

In [53]:
z_pred

array(['Fireworks', 'Fireworks', 'Fireworks'], dtype=object)

In [54]:
y

array([-6.9934599e-06, -1.3333531e-04,  2.5686712e-04, ...,
        4.6400787e-06, -3.3317032e-05, -1.3393701e-05], dtype=float32)

In [55]:
type(X_test)

pandas.core.frame.DataFrame

In [56]:
type(Z)

pandas.core.frame.DataFrame